In [1]:
##################################################################
# Purpose: Create a 3D image using the published Mopra dataset
# Author:  G. Wong
# Date:    15.10.21
#
# Notes:   This is an update from a previous version from 2015.
#          Suitable for python 3.7 only.
#
#          package name change - image now PIL (Pillow)
#                              - pyfits integrated into astropy.fits
#
##################################################################

In [2]:
# System Checks
import sys
print (sys.version)

3.7.10 | packaged by conda-forge | (default, Oct 13 2021, 20:22:05) [MSC v.1916 64 bit (AMD64)]


In [3]:
#conda activate python3.7   # to be used within jupyter notebook python 3.7

In [4]:
# import libraries
from astropy.io import fits
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D    # used in matplotlib
import numpy as np
import scipy.ndimage as ndimage

from PIL import Image                # used to export the PS files


from mayavi import mlab
from mayavi.mlab import *


In [5]:
# Parameters
reverselut=False

FITSfilename = "G330.5/G330.5-12CO_3sig_vcrop.fits"  #Blues     #The name of the file to read in.
contours = np.arange(6,12,2)		#Specify the contour levels. The numbers are start, end, stepsize
colourscale = "Blues"
colourlimits = [1,10]     #min,max of colourbar
colour = (0,0,1)
print('Contour levels =', contours)

#framesize = [1280,680]   
framesize = [1280,720]  


Contour levels = [ 6  8 10]


In [6]:
# Extract header
fitsfile = fits.open(FITSfilename)
data_cube = fitsfile[0].data
header = fitsfile[0].header
fitsfile.close()

In [7]:
# Extract header infomration
crpix1 = header['CRPIX1']		#reference pixel 
cdelt1 = header['CDELT1']		#step size
crval1 = header['CRVAL1']		#coordinate for reference pixel
ctype1 = header['CTYPE1']

crpix2 = header['CRPIX2']		#latitude
cdelt2 = header['CDELT2']
crval2 = header['CRVAL2']
ctype2 = header['CTYPE2']

crpix3 = header['CRPIX3']		#velocity
cdelt3 = header['CDELT3']
crval3 = header['CRVAL3']	
ctype3 = header['CTYPE3']

bunit = header['BUNIT']

In [8]:
# Data formatting for visualisation, create a blank cube and fill it with the data
print(data_cube[0,0,0], type(data_cube[0,0,0]))
print(np.nanmean(data_cube))
#data_cube = data_cube*1.0 			#sometimes the formatting is wrong, so multiply everything by 1 to fix.
data_cube = data_cube.astype(np.float)

#print data_cube.type
print(data_cube[0,0,0], type(data_cube[0,0,0]))
print(np.nanmean(data_cube))
print('Data cube axes =', ctype3, ctype2, ctype1)
print('Data cube size = ', data_cube.shape)

vel = [((x+1-crpix3)*cdelt3+crval3)/1000 for x in range(data_cube.shape[0])]		#+1 because the FITS pixels are 1 to n, the python indexes are 0 to (n-1)
glat = [(x+1-crpix2)*cdelt2+crval2 for x in range(data_cube.shape[1])]
glon = [(x+1-crpix1)*cdelt1+crval1 for x in range(data_cube.shape[2])]

data_cube = data_cube[::-1,:,:] 			#Reverse the velocity axis, so that positive velocity is further away. Maybe not necessary?
#data_cube = data_cube[39:172,8:126,10:600]		#If you want to cut some edges off the cube, you can specify it here
vel = vel[1:1700]
glat = glat[1:122]
glon = glon[1:122]


vel = vel[::-1]

print('Axes size =', len(vel), len(glat), len(glon))
print('Axes limits =', [vel[0],vel[-1]], [glat[0],glat[-1]], [glon[0],glon[-1]])

print(data_cube[0,0,0], type(data_cube[0,0,0]))
data_cube[np.isnan(data_cube)]= 0 			#set any nan values to zero
print(data_cube[0,0,0], type(data_cube[0,0,0]))

data_cube = ndimage.gaussian_filter(data_cube, sigma=(0.5, 0.5, 0.5), order=0) 			#apply a gaussian filter to smooth the data. sigma = standard deviation for each axis

data_cube=np.transpose(data_cube,(0,2,1))   #sort into velocity, longitude, latitude order. Might depend on your fits file


nan <class 'numpy.float32'>
8.477997
nan <class 'numpy.float64'>


C:\Users\gwong\anaconda3\envs\python3.7\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


8.478003629037465
Data cube axes = VELO-LSR GLAT-CAR GLON-CAR
Data cube size =  (1701, 122, 122)
Axes size = 1699 121 121
Axes limits = [39.9, -129.9] [-0.493750000000395, 0.506250000000405] [330.9937499999844, 329.9937499999836]
nan <class 'numpy.float64'>
0.0 <class 'numpy.float64'>


In [9]:
# Imaging 

mlab.figure(size=framesize)

scalarfield = mlab.pipeline.scalar_field(data_cube)		#create a scalar field object from the data cube
print(scalarfield, type(scalarfield))
scalarfield.spacing = [1,1,1]							#stretch out the velocity axis.

#contours = np.arange(0.5,np.amax(data_cube),np.amax(data_cube)/5.0)	
#print 'Contour levels =', contours

#surf = mlab.pipeline.iso_surface(scalarfield, contours=contours.tolist(), opacity=0.3, colormap=colourscale)  #, extent=[0,180,3,50,3,50])	#create a surface object from the scalar field
vol = mlab.pipeline.volume(scalarfield, vmin=contours[0],vmax=contours[1],color=colour)


#lut1 = surf.module_manager.scalar_lut_manager.lut.table.to_array()
#print lut1
#surf.module_manager.scalar_lut_manager.lut.table = lut
#surf = mlab.contour3d(data_cube)

#min = data_cube.min()
#max = data_cube.max()


mlab.figure(figure = mlab.gcf(),bgcolor=(1,1,1),fgcolor = (0,0,0))

f = mlab.outline()
f.outline_mode='cornered'
f.bounds = [0,180,0,47,0,47]
#surf.extent = [0,200,3,50,3,50]

#surf.module_manager.scalar_lut_manager.data_range = [colourlimits[0], colourlimits[1]]
#surf.module_manager.scalar_lut_manager.reverse_lut = reverselut
vol.module_manager.scalar_lut_manager.data_range = [colourlimits[0], colourlimits[1]]
vol.module_manager.scalar_lut_manager.data_range = [contours[0], contours[-1]]
vol.module_manager.scalar_lut_manager.reverse_lut = reverselut





<mayavi.sources.array_source.ArraySource object at 0x000001ED2AAF4EE8> <class 'mayavi.sources.array_source.ArraySource'>


In [10]:
# Image settings

ti = mlab.title(FITSfilename[5:-11] + '.',height=0.9, size = 0.3)
ti.y_position = 0.9
#ax = mlab.axes(xlabel=ctype3, ylabel=ctype1, zlabel=ctype2, ranges =[vel[0],vel[-1],glon[0],glon[-1],glat[0],glat[-1]])
ax = mlab.axes(xlabel='Vel', ylabel='Lon', zlabel='Lat',ranges =[vel[0],vel[-1],glon[0],glon[-1],glat[0],glat[-1]])
#ax = mlab.axes(xlabel='Vel', ylabel='Lon', zlabel='Lat', ranges =[0,-80,328.74,327.86,-0.52,0.52])
#ax.bounds = [0,180,3,50,3,50]
print(vel[0],vel[-1],glon[0],glon[-1],glat[0],glat[-1])

ax.axes.label_format='%-6.3g'
ax.title_text_property.font_family = 'times'
ax.title_text_property.font_size = 20
ax.axes.font_factor=1
ax.property.opacity = 0.0
ax.label_text_property.opacity = 0.0 			#set to 0 to hide text labels
ax.axes.corner_offset=0
ax.title_text_property.line_offset = 0.0
ax.title_text_property.vertical_justification = 'centered'
ax.title_text_property.justification = 'centered'
ax.title_text_property.bold = False


39.9 -129.9 330.9937499999844 329.9937499999836 -0.493750000000395 0.506250000000405


In [11]:
# Camera orientation

#mlab.orientation_axes(xlabel='Vel', ylabel='Lon', zlabel='Lat')
f = mlab.gcf()
#f.scene.camera.parallel_projection = False
#mlab.colorbar(title=bunit)
#view(220,70,distance=1500,focalpoint="auto")
#view(50,90,distance=190,focalpoint="auto")
#camera_light3 = engine.scenes[0].scene.light_manager.lights[3]
#camera_light3.activate = True
#camera_light3.intensity = 0.5

# 2021 no attributes lights
#f.scene.light_manager.lights[3].activate = True
#f.scene.light_manager.lights[3].intensity = 0.5


view(60,50,distance=2000,focalpoint=[1000, 50,50])
print('Current view =', view())
#mlab.move(0,-20,0)
#print('Current view =', view())



mlab.savefig(FITSfilename + 'vol.jpeg')
mlab.savefig(FITSfilename + 'vol.ps')

#mlab.show()



# image12CO = Image.open('12COvol.ps')
# image13CO = Image.open('13COvol.ps')
# imageCI = Image.open('CIvol.ps')
# imageHI = Image.open('HIvol.ps')


blank_image = Image.new("RGB", (framesize[0]*2, framesize[1]*2-60))
#blank_image.paste(image12CO, (0,0))
#blank_image.paste(image13CO, (framesize[0],0))
#blank_image.paste(imageCI, (0,framesize[1]-30))
#blank_image.paste(imageHI, (framesize[0],framesize[1]-30))
# blank_image.save('cubess120vol.ps')

#blank_image.save('cubess120.eps')

mlab.show()

#click "Stop Interaction", then close the window.

Current view = (59.99999999999999, 50.0, 1999.9999999999998, array([1000.,   50.,   50.]))


In [12]:
#To create an interactive HTML, you'll need to edit cloud.x3d with a text editor.

#change the 4th DirectionalLight to on="true"
#add <DepthMode readOnly="true"></DepthMode> under <Appearance>		(most important!)
#near the bottom, change emissiveColor for the frame to "1 1 1" if you want a white outline
#change the title position and rotation. I used        <Transform translation="120 25 60" rotation="0 1 1 3.14159" scale="1 1 1" >
#change the title to not solid  <Text string="name.fits" solid="False">
#add in any extra labels after </Collision> in the title section. for example:
# <Transform translation="250 25 -5" rotation="-1 -1 -1 -2.0944" scale="1 1 1" >
#   <Shape>
#     <Appearance>
#       <Material diffuseColor="0 0 1" emissiveColor="1 1 1"/>
#     </Appearance>
#     <Text string="GLon" solid="False">
#       <FontStyle family='"SANS"' justify='"MIDDLE" "MIDDLE"' size="8"/>
#     </Text>
#   </Shape>
# </Transform>